# CHAPTER 08. 텍스트 분석

## 02. 텍스트 사전 준비 작업(텍스트 전처리) - 텍스트 정규화
- **텍스트 정규화** : 머신러닝 알고리즘이나 NLP 어플리케이션에 입력 데이터로 사용하기 위해 클렌징, 정제, 토큰화, 어근화 등의 다양한 텍스트 데이터의 사전 작업을 수행하는 것. 텍스트 분석에서 이러한 텍스트 정규화 작업이 매우 중요하다.

### < 텍스트 정규화 작업 >
### 1. 클렌징
: 텍스트에서 분석에 오히려 방해가 되는 불필요한 문자, 기호 등을 사전에 제거하는 작업

### 2. 텍스트 토큰화 
#### 1) 문장 토큰화
: 문장의 마침표(.), 개행문자(\n) 등 문장의 마지막을 뜻하는 기호에 따라 분리하는 것이 일반적이다. 또한 정규 표현식에 따른 문장 토큰화도 가능하다. 


각 문장이 가지는 시맨틱적인 의미가 중요한 요소로 사용될 때 사용한다 ! ! 
- **sent_tokenize** : NLTK에서 일반적으로 많이 쓰이는 API

In [9]:
# 예제
from nltk import sent_tokenize
import nltk
nltk.download('punkt')

text_sample = 'The Matrix is everywhere its all around us, here even in this room. \
               You can see it out your window or on your television. \
               You feel it when you go to work, or go to church or pay your taxes.'
sentences = sent_tokenize(text=text_sample)
print(type(sentences), len(sentences))
print(sentences)

<class 'list'> 3
['The Matrix is everywhere its all around us, here even in this room.', 'You can see it out your window or on your television.', 'You feel it when you go to work, or go to church or pay your taxes.']


[nltk_data] Downloading package punkt to /Users/wizdom/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


sent_tokenize()가 반환하는 것은 각각의 문장으로 구성된 list 객체이다.

#### 2) 단어 토큰화
: 문장을 단어로 토큰화하는 것이다. 기본적으로 공백, 콤마, 마침표, 개행문자 등으로 단어를 분리하지만, 정규 표현식을 이용해 다양한 유형으로 토큰화를 수행할 수 있다. 


단어의 순서가 중요하지 않은 경우 문장 토큰화를 사용하지 않고 단어 토큰화만 사용한다.
- **word_tokenize** : NLTK에서 기본으로 제공하는 단어 토큰화 API


In [10]:
# 예제
from nltk import word_tokenize

sentence = "The Matrix is everywhere its all around us, here even in this room."
words = word_tokenize(sentence)
print(type(words), len(words))
print(words)

<class 'list'> 15
['The', 'Matrix', 'is', 'everywhere', 'its', 'all', 'around', 'us', ',', 'here', 'even', 'in', 'this', 'room', '.']


In [11]:
# sent_tokenize와 word_tokenize를 조합해 문서에 대해 모든 단어 토큰화 하기

from nltk import word_tokenize, sent_tokenize

# 여러 개의 문장으로 된 입력 데이터를 문장별로 단어 토큰화하게 만드는 함수 생성
def tokenize_text(text):
    
    # 문장별로 분리 토큰
    sentences = sent_tokenize(text)
    # 분리된 문장별 단어 토큰화
    word_tokens = [word_tokenize(sentence) for sentence in sentences]
    return word_tokens

# 여러 문장에 대해 문장별 단어 토큰화 수행
word_tokens = tokenize_text(text_sample)
print(type(word_tokens), len(word_tokens))
print(word_tokens)

<class 'list'> 3
[['The', 'Matrix', 'is', 'everywhere', 'its', 'all', 'around', 'us', ',', 'here', 'even', 'in', 'this', 'room', '.'], ['You', 'can', 'see', 'it', 'out', 'your', 'window', 'or', 'on', 'your', 'television', '.'], ['You', 'feel', 'it', 'when', 'you', 'go', 'to', 'work', ',', 'or', 'go', 'to', 'church', 'or', 'pay', 'your', 'taxes', '.']]


3개의 리스트 객체를 내포하는 리스트가 출력되었다. 그리고 내포된 개별 리스트 객체는 각각 문장별로 토큰화된 단어를 요소로 가지고 있다.

문장을 단어별로 하나씩 토큰화 할 경우 문맥적인 의미는 무시될 수 밖에 없다. 이러한 문제를 조금이라도 해결해 보고자 도입 된 것이 n-gram이다.
- **n_gram** : 연속된 n개의 단어를 하나의 토큰화 단위로 분리해 내는 것이다. n개 단어 크기 윈도우를 만들어 문장의 처음부터 오른쪽으로 움직이면서 토큰화를 수행한다. 
	- 예) 2-gram(bigram)으로 만들기
	- “Agent Smith knocks the door”
	- -> (Agent, Smith), (Smith, knocks), (knocks, the), (the, door)
	- 이와 같이 연속적으로 2개의 단어들을 순차적으로 이동하면서 단어들을 토큰화한다.

### 스톱 워드 제거
: 분석에 큰 의미가 없는 단어를 지칭한다. (영어에서 is, the, a will 등 문장을 구성하는 필수 문법 요소지만 문맥적으로 큰 의미가 없는 단어)
- 이 단어의 경우 문법적인 특성으로 인해 특히 빈번하게 텍스트에 나타나므로 이것들을 사전에 제거하지 않으면 그 빈번함으로 인해 오히려 중요한 단어로 인지될 수 있다. 따라서 이 단어들을 제거하는 것이 중요한 전처리 작업이다.
- 언어별로 스톱 워드가 목록화돼 있다. NLTK 가 가장 다양하게 제공

In [12]:
import nltk
nltk.download('stopwords')

print('영어 stop words 개수:', len(nltk.corpus.stopwords.words('english')))
print(nltk.corpus.stopwords.words('english')[:20])

영어 stop words 개수: 179
['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his']


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/wizdom/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [13]:
# word_tokens 리스트에 대해 stopwords를 필터링으로 제거해 분석을 위한 의미있는 단어만 추출

import nltk

stopwords = nltk.corpus.stopwords.words('english')
all_tokens = []
# 위 예제의 3개의 문장별로 얻은 word_tokens list 에 대해 stop word 제거 Loop
for sentence in word_tokens:
    filtered_words=[]
    # 개별 문장별로 tokenize된 sentence list에 대해 stop word 제거 Loop
    for word in sentence:
        #소문자로 모두 변환
        word = word.lower()
        # tokenize 된 개별 word가 stop words 들의 단어에 포함되지 않으면 word_tokens에 추가
        if word not in stopwords:
            filtered_words.append(word)
    all_tokens.append(filtered_words)
    
print(all_tokens)

[['matrix', 'everywhere', 'around', 'us', ',', 'even', 'room', '.'], ['see', 'window', 'television', '.'], ['feel', 'go', 'work', ',', 'go', 'church', 'pay', 'taxes', '.']]


is, this와 같은 스톱워드가 필터링을 통해 제거 됐음을 알 수 있다.

### Stemming과 Lemmatization
: 문법적으로 또는 의미적으로 변화하는 단어의 원형을 찾는 것
- Lemmatization이 Stemming보다 정교하며 의미론적인 기반에서 단어의 원형을 찾는다.


- 대표적 Stemmer : Porter, Lancaster, Snowball Stemmer
- Lemmatization을 위해서는 WordNetLemmatizer 사용

#### Stemming과 Lemmatization 비교

- **LancasterStemmer 이용**
    - LancasterStemmer()와 같이 필요한 Stemmer 객체를 생성한 뒤 이 객체의 stem(‘단어’) 메서드를 호출하면 원하는 ‘단어’의 Stemming이 가능|

In [14]:
# LancasterStemmer를 이용해 Stemmer 살펴보기

from nltk.stem import LancasterStemmer
stemmer = LancasterStemmer()

print(stemmer.stem('working'),stemmer.stem('works'),stemmer.stem('worked'))
print(stemmer.stem('amusing'),stemmer.stem('amuses'),stemmer.stem('amused'))
print(stemmer.stem('happier'),stemmer.stem('happiest'))
print(stemmer.stem('fancier'),stemmer.stem('fanciest'))

work work work
amus amus amus
happy happiest
fant fanciest


단순하게 원형 단어를 찾는다. 따라서 amuse와 형용사인 happy, fancy의 경우 단어의 정확한 원형을 찾지 못하는 것을 볼 수 있다.

- **WordNetLemmatizer 이용**
	- Lemmatization은 보다 정확한 원형 단어 추출을 위해 단어의 **품사**를 입력해주어야 한다.

In [15]:
# WordNetLemmatizer를 이용한 LEmmatization 수행
from nltk.stem import WordNetLemmatizer
import nltk
nltk.download('wordnet')

lemma = WordNetLemmatizer()
print(lemma.lemmatize('amusing','v'),lemma.lemmatize('amuses','v'),lemma.lemmatize('amused','v'))
print(lemma.lemmatize('happier','a'),lemma.lemmatize('happiest','a'))
print(lemma.lemmatize('fancier','a'),lemma.lemmatize('fanciest','a'))

[nltk_data] Downloading package wordnet to /Users/wizdom/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


amuse amuse amuse
happy happy
fancy fancy


앞의 Stemmer보다 정확하게 원형 단어를 추출해줌을 알 수 있다.

## 03. Bag of Words - BOW
: 문서가 가지는 모든 단어를 문맥이나 순서를 무시하고 일괄적으로 단어에 대해 빈도 값을 부여해 피처 값을 추출하는 모델이다. 

- 장점 
	- 쉽고 빠른 구축. 문서의 특징을 잘 나타낼 수 있는 모델이라 전통적으로 여러 분야에서 활용도가 높다.
- 단점 
	- **문맥 의미(Semantix Context) 반영 부족** : BOW는 단어의 순서를 고려하지 않기 때문에 문장 내에서 단어의 문맥적인 의미가 무시된다. 
	- **희소 행렬 문제(희소성, 희소 행렬)** : BOW로 피처 벡터화를 수행하면 희소 행렬 형태의 데이터 세트가 만들어지기 쉽다. 

### BOW 피처 벡터화
- **피처 백터화** : 텍스트를 특정의미를 가지는 숫자형 값인 벡터 값으로 변환해주는 것

### 희소 행렬 - COO 형식
: 0이 아닌 데이터만 별도의 데이터 배열에 저장하고, 그 데이터가 가리키는 행과 열의 위치를 별도의 배열로 저장하는 방식
- 희소 행렬 변환을 위해 사이파이를 이용한다. **사이파이의 sparse 패키지**는 희소 행렬 변환을 위해 다양한 모듈을 제공한다.

In [16]:
# 넘파이 ndarray 객체로 만들기

import numpy as np

dense = np.array([[3, 0 , 1], [0, 2, 0]])

In [ ]:
# 위 밀집행렬을 COO 형식의 희소 행렬로 변환
from scipy import sparse

# 0이 아닌 ㄷㅔ이터 ㅊㅜ출